In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


To install dependencies

In [6]:
!pip install norfair
!pip install nvidia-tensorrt==8.4.1.5
!pip install onnx onnxruntime-gpu
!pip install ultralytics
!pip install numpy==1.23.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.6/607.6 kB 6.5 MB/s eta 0:00:00


Milestone 1 (Player detection using the YOLOv8 models trained on COCO dataset)

In [8]:
import sys
root_path = '/content/drive/MyDrive/basketball-broadcaster'
sys.path.append(root_path)

import os
os.chdir(root_path)

import cv2
from inference.yolov8 import YOLOv8
from norfair import Video
import time

from ultralytics import YOLO

model = YOLO("yolov8m.pt")
model.export(format="onnx")
model = YOLO("yolov8l.pt")
model.export(format="onnx")
model = YOLO("yolov8x.pt")
model.export(format="onnx")

Ultralytics YOLOv8.0.141 🚀 Python-3.10.6 torch-2.0.1+cu118 CPU (Intel Xeon 2.30GHz)
YOLOv8m summary (fused): 218 layers, 25886080 parameters, 0 gradients

PyTorch: starting from 'yolov8m.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (49.7 MB)

ONNX: starting export with onnx 1.14.0 opset 17...


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



ONNX: export success ✅ 5.0s, saved as 'yolov8m.onnx' (99.0 MB)

Export complete (9.4s)
Results saved to /content/drive/MyDrive/basketball-broadcaster
Predict:         yolo predict task=detect model=yolov8m.onnx imgsz=640 
Validate:        yolo val task=detect model=yolov8m.onnx imgsz=640 data=coco.yaml 
Visualize:       https://netron.app
100%|██████████| 83.7M/83.7M [00:02<00:00, 43.7MB/s]
Ultralytics YOLOv8.0.141 🚀 Python-3.10.6 torch-2.0.1+cu118 CPU (Intel Xeon 2.30GHz)
YOLOv8l summary (fused): 268 layers, 43668288 parameters, 0 gradients

PyTorch: starting from 'yolov8l.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (83.7 MB)

ONNX: starting export with onnx 1.14.0 opset 17...


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



ONNX: export success ✅ 7.5s, saved as 'yolov8l.onnx' (166.8 MB)

Export complete (15.4s)
Results saved to /content/drive/MyDrive/basketball-broadcaster
Predict:         yolo predict task=detect model=yolov8l.onnx imgsz=640 
Validate:        yolo val task=detect model=yolov8l.onnx imgsz=640 data=coco.yaml 
Visualize:       https://netron.app
100%|██████████| 131M/131M [00:00<00:00, 144MB/s]
Ultralytics YOLOv8.0.141 🚀 Python-3.10.6 torch-2.0.1+cu118 CPU (Intel Xeon 2.30GHz)
YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients

PyTorch: starting from 'yolov8x.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (130.5 MB)

ONNX: starting export with onnx 1.14.0 opset 17...


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



ONNX: export success ✅ 8.6s, saved as 'yolov8x.onnx' (260.4 MB)

Export complete (24.2s)
Results saved to /content/drive/MyDrive/basketball-broadcaster
Predict:         yolo predict task=detect model=yolov8x.onnx imgsz=640 
Validate:        yolo val task=detect model=yolov8x.onnx imgsz=640 data=coco.yaml 
Visualize:       https://netron.app


'yolov8x.onnx'

In [9]:
model_paths = ['./yolov8m.onnx', './yolov8l.onnx', './yolov8x.onnx']
model_level = 0
player_detector = YOLOv8(path=model_paths[model_level], conf_thres=0.3, iou_thres=0.3)

In [ ]:
def draw_player_detections(image, boxes, scores, class_ids):
    det_img = image.copy()

    img_height, img_width = image.shape[:2]
    size = min([img_height, img_width]) * 0.0006
    text_thickness = int(min([img_height, img_width]) * 0.001)

    # Draw bounding boxes and labels of detections
    for box, score, class_id in zip(boxes, scores, class_ids):
        if class_id != 0:
            continue

        color = (0, 255, 0)

        x1, y1, x2, y2 = box.astype(int)

        # Draw rectangle
        cv2.rectangle(det_img, (x1, y1), (x2, y2), color, 2)

        caption = '%.2f' % (score)
        (tw, th), _ = cv2.getTextSize(text=caption, fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                                      fontScale=size, thickness=text_thickness)
        th = int(th * 1.2)

        cv2.rectangle(det_img, (x1, y1),
                      (x1 + tw, y1 - th), color, -1)
        cv2.putText(det_img, caption, (x1, y1),
                    cv2.FONT_HERSHEY_SIMPLEX, size, (255, 255, 255), text_thickness, cv2.LINE_AA)

    return det_img

video = Video(input_path='videos/test1_clip.mp4', output_fps=30)

frame_cnt = 0
total_time = 0
for i, frame in enumerate(video):
    st = time.time()
    boxes, scores, class_ids = player_detector(frame)
    elapsed = time.time() - st

    total_time += elapsed
    fps = frame_cnt // total_time

    frame = draw_player_detections(frame, boxes, scores, class_ids)

    h, w = frame.shape[:2]
    cv2.putText(frame, 'FPS: {}'.format(fps), (100, 50), cv2.FONT_HERSHEY_SIMPLEX,
                1.0, (255, 0, 0), 2, cv2.LINE_AA)

    frame_cnt += 1
    video.write(frame)